In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from transformers import pipeline
import torch

In [2]:
import os
import pandas as pd
import librosa

def obtener_nombres_archivos(ruta):
    archivos = []
    for nombre_archivo in os.listdir(ruta):
        if os.path.isfile(os.path.join(ruta, nombre_archivo)):
            ruta_completa = os.path.join(ruta, nombre_archivo)
            duracion = obtener_duracion_audio(ruta_completa)
            archivos.append({'NombreArchivo': nombre_archivo, 'RutaArchivo': ruta_completa, 'Duracion': duracion})
    df = pd.DataFrame(archivos)
    return df

def obtener_duracion_audio(ruta_archivo):
    audio, sr = librosa.load(ruta_archivo)
    duracion = len(audio) / sr
    return duracion

ruta_carpeta = r'D:\sentirobots\Audios para pruebas'
df = obtener_nombres_archivos(ruta_carpeta)
df

,NombreArchivo,RutaArchivo,Duracion
0,00038.wav,D:\sentirobots\Audios para pruebas\00038.wav,5.500000
1,000451774.wav,D:\sentirobots\Audios para pruebas\000451774.wav,2.184036
2,000452240.wav,D:\sentirobots\Audios para pruebas\000452240.wav,3.288027
3,000505440.wav,D:\sentirobots\Audios para pruebas\000505440.wav,2.760000
4,000513320.wav,D:\sentirobots\Audios para pruebas\000513320.wav,1.776009
...,...,...,...
71,video44.wav,D:\sentirobots\Audios para pruebas\video44.wav,259.530159
72,video51.wav,D:\sentirobots\Audios para pruebas\video51.wav,146.913379
73,video60.wav,D:\sentirobots\Audios para pruebas\video60.wav,63.994875
74,video67.wav,D:\sentirobots\Audios para pruebas\video67.wav,61.649660


In [3]:
import pandas as pd
import librosa
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

sample_rate = 22400
max_time = 7.

traindf = pd.DataFrame(columns=['filename', 'Duration', 'Text'])

transcription_model = Wav2Vec2ForCTC.from_pretrained("jonatasgrosman/wav2vec2-xls-r-1b-english")
tokenizer = Wav2Vec2Tokenizer.from_pretrained("jonatasgrosman/wav2vec2-xls-r-1b-english")

def normalize_audio(audio):
    normalized = (audio - audio.mean()) / audio.std()
    return normalized

def transcribe_audio(audio, model):
    input_values = tokenizer(audio, return_tensors="pt").input_values
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = tokenizer.decode(predicted_ids[0])
    return transcription

for _, row in tqdm(df.iterrows()):
    audiofile, _ = librosa.load(path=row['RutaArchivo'], sr=sample_rate, duration=np.minimum(row['Duracion'], max_time))

    if row['Duracion'] < max_time:
        audiofile = librosa.util.pad_center(data=audiofile, size=sample_rate * max_time)

    normalized = normalize_audio(audiofile)
    transcription = transcribe_audio(normalized, transcription_model)

    rowdf = pd.DataFrame({
        'filename': row['RutaArchivo'],
        'Duration': row['Duracion'],
        'Text': transcription,
    }, index=pd.Index([0]))

    traindf = pd.concat([traindf, rowdf], ignore_index=True)

traindf


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
C:\ProgramData\Anaconda3\lib\site-packages\transformers\models\wav2vec2\tokenization_wav2vec2.py:757: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(
76it [08:59,  7.10s/it]


,filename,Duration,Text
0,D:\sentirobots\Audios para pruebas\00038.wav,5.500000,ande feed like all indulge you curiou steel no...
1,D:\sentirobots\Audios para pruebas\000451774.wav,2.184036,now before this i was halp
2,D:\sentirobots\Audios para pruebas\000452240.wav,3.288027,i need to get into grangos into one of the volds
3,D:\sentirobots\Audios para pruebas\000505440.wav,2.760000,you are not dancing you are not dancing ino
4,D:\sentirobots\Audios para pruebas\000513320.wav,1.776009,happy thirty et
...,...,...,...
71,D:\sentirobots\Audios para pruebas\video44.wav,259.530159,hala my name is danny and this is my extremely...
72,D:\sentirobots\Audios para pruebas\video51.wav,146.913379,untempted to say smart creative people have no
73,D:\sentirobots\Audios para pruebas\video60.wav,63.994875,the mayno elek poter sohi dars there is a tuwe...
74,D:\sentirobots\Audios para pruebas\video67.wav,61.649660,hils ti first twee d


In [4]:
columna = "Text"

num_vacios = traindf[columna].eq('').sum()
total_registros = traindf[columna].shape[0]
porcentaje_vacios = (num_vacios / total_registros) * 100

print("Número de valores vacíos en la columna {}: {}".format(columna, num_vacios))
print("Total de registros en la columna {}: {}".format(columna, total_registros))
print("Porcentaje de valores vacíos en la columna {}: {:.2f}%".format(columna, porcentaje_vacios))

Número de valores vacíos en la columna Text: 4
Total de registros en la columna Text: 76
Porcentaje de valores vacíos en la columna Text: 5.26%


In [6]:
traindf.to_excel(r'D:\sentirobots\Audios para pruebas\transcripcion-jonatasgrosman-wav2vec2.xlsx', index=False)
print("Realizado")

Realizado
